<div style="
  padding: 30px;
  text-align: center;" class='row'>
<div style="float:left;width: 15%;" class='column'><a href="https://www.colombiacompra.gov.co"><img alt="Logo Colombia Compra Eficiente" id="logocce" src="https://www.colombiacompra.gov.co/sites/cce_public/files/Images/logocce.png" style="height: 45px;"></a></div>
    <div style="float:left;width: 70%;" class='column'>
        <h1> Reporte rutas y modificaciones
        </h1> 
    </div>
<div style="float:left;width: 15%;" class='column'><a href="https://www.dnp.gov.co/" target="_blank"><img class="float-right" id="logodnp" src="https://www.dnp.gov.co/img/logoNuevo.jpg" style="width: 200px;"></a></div>
</div>

El objetivo principal de este cuadernillo es proporcionar una guía paso a paso para obtener información relevante sobre las versiones del proyecto y almacenarla en un formato estructurado. En particular, se generará un archivo Excel que contendrá los siguientes detalles:

* Ruta del archivo: Se registrarán las rutas de los archivos en el Data Lake, lo que permitirá identificar su ubicación exacta en el sistema.

* Última modificación: Se obtendrá la fecha y hora de la última modificación realizada en cada archivo. Esto ayudará a tener un seguimiento actualizado de los cambios realizados en las versiones.

* Tamaño del archivo: Se proporcionará el tamaño de cada archivo en diferentes unidades, como megabytes (MB), mebibytes (MiB), gigabytes (GB) y gibibytes (GiB). Esto facilitará el análisis de los recursos utilizados por cada versión.

### Librerías

In [1]:
# Personalizada de inicio de sesión
from libreria import login

# Herramientas
import pandas as pd
import datetime

## Obtención de información sobre archivos en el Data Lake

En esta sección del cuadernillo, se realiza la obtención de información sobre los archivos almacenados en el Data Lake. El código se encarga de obtener la lista de blobs (archivos) en el contenedor especificado.

In [2]:
# Se establece la conexión al Data Lake
blob_service_client, container_client = login.login_datalake()

# Lista vacía que será utilizada para almacenar la información de los archivos.
control_de_versiones = []

blobs = container_client.list_blobs()
for blob in blobs:
    # se obtiene el nombre del archivo y su tamaño
    file_name = blob.name
    file_size = blob.size

    # Obtener la información de la última modificación del archivo
    blob_client = blob_service_client.get_blob_client(container=login.cliente_contenedor.datosemae, blob=file_name)
    properties = blob_client.get_blob_properties()
    last_modified = pd.Timestamp(properties.last_modified)
    last_modified_by = properties.metadata.get('last_modified_by')

    control_de_versiones.append({
        'archivo': file_name, 
        'tamaño mb': str(round(float(file_size)*0.000001, 2)) + " mb", # Conversión a mb
        'tamaño mib': str(round(float(file_size)*0.00000095367, 2)) + " mib", # Conversión a mib
        'tamaño gb': str(round(float(file_size)*0.000000001, 2)) + " gb", # Conversión a gb
        'tamaño gib': str(round(float(file_size)*0.00000000093132, 2)) + " gib", # Conversión a gib
        'ultima modificacion': last_modified.tz_convert(None), 
        'modificado por': last_modified_by
    })

control_de_versiones = pd.DataFrame(control_de_versiones)

### Exportación de datos a Excel

El código también incluye una función para exportar la lista control_de_versiones a un archivo de Excel. El archivo se guarda en la carpeta de reportes con un nombre que incluye la fecha actual.

In [3]:
control_de_versiones.to_excel(f'../reportes/versiones/version {datetime.datetime.now().date()}.xlsx')